In [1]:
# CS 598 HW1
# Student: Zhi Ji, code is a modified version of professor Sirigano's work
import numpy as np
import os
import h5py
import time
import copy
from random import randint

#load MNIST dat
MNIST_data = h5py.File('MNISTdata.hdf5', 'r')
x_train = np.float32(MNIST_data['x_train'][:] )
y_train = np.int32(np.array(MNIST_data['y_train'][:,0]))
x_test = np.float32( MNIST_data['x_test'][:] )
y_test = np.int32( np.array( MNIST_data['y_test'][:,0] ) )
MNIST_data.close()

In [2]:
#Implementation of stochastic gradient descent algorithm
#number of inputs
num_inputs = 28*28
#number of hidden units
num_h = 100
#number of outputs
num_outputs = 10

#initialize parameters
model = {}
model['W1'] = np.random.randn(num_h,num_inputs) / np.sqrt(num_inputs)
model['b1'] = np.zeros((num_h,1))
model['b2'] = np.zeros((num_outputs,1))
model['C'] = np.random.randn(num_outputs,num_h) / np.sqrt(num_inputs)
model_grads = copy.deepcopy(model)
#print(model['W1'].shape)
#print(model['b1'].shape)
#print(model['C'].shape)
#print(model['b2'].shape)

def softmax_function(z):
    ZZ = np.exp(z)/np.sum(np.exp(z))
    return ZZ

def sigmoid(z):
    return np.exp(z)/(1+np.exp(z))

def forward(x,y, model):
    #print(x.reshape((784,1)).shape)
    x = x.reshape((784,1))
    Z = np.dot(model['W1'], x) + model['b1']
    #print((model['W1'].dot(x)+model['b1']).shape)
    #print(model['b1'].shape)
    #print(Z.shape)
    H = sigmoid(Z)
    model['H'] = H
    model['Z'] = Z
    U = np.dot(model['C'],H) + model['b2']
    p = softmax_function(U)
    return p

#derivative of sigmoid
def sigmoid_back(z):
    s = 1/(1+np.exp(-z))
    return s * (1-s)
    
def backward(x,y,p, model, model_grads):
    dU = -1.0*p
    dU[y] = dU[y] + 1.0
    db2 = dU
    dC = np.dot(dU,model['H'].T)
    delta = np.dot(model['C'].T,dU)
    sigmaprime = sigmoid_back(model['Z'])
    db1 = np.multiply(delta,sigmaprime)
    #print(delta.shape)
    #print(db1.shape)
    #print(sigmaprime.shape)
    #print(x.shape)
    temp = x.shape[0]
    dW1 = np.dot(db1,x.reshape((1,temp)))
    
    #for i in range(num_outputs):
        #model_grads['W1'][i,:] = dZ[i]*x
        
        #model_grads['b1'][i,:] = dZ[i]*x
        #model_grads['C'][i,:] =  dZ[i]*x
        #model_grads['b2'][i,:] = dZ[i]*x
    model_grads['W1'] = dW1  
    model_grads['b1'] = db1
    model_grads['C'] =  dC
    model_grads['b2'] = db2
        
    return model_grads


In [3]:
#timer
import time
time1 = time.time()
LR = .01
num_epochs = 40

for epochs in range(num_epochs):
    #Learning rate schedule
    if (epochs > 5):
        LR = 0.001
    if (epochs > 10):
        LR = 0.0001
    if (epochs > 15):
        LR = 0.00001
    total_correct = 0
    for n in range( len(x_train)):
        n_random = randint(0,len(x_train)-1 )
        y = y_train[n_random]
        x = x_train[n_random][:]
        p = forward(x, y, model)
        prediction = np.argmax(p)
        if (prediction == y):
            total_correct += 1
        model_grads = backward(x,y,p, model, model_grads)
        
        #update parameter
        model['W1'] = model['W1'] + LR*model_grads['W1']
        model['b1'] = model['b1'] + LR*model_grads['b1']
        model['C'] = model['C'] + LR*model_grads['C']
        model['b2'] = model['b2'] + LR*model_grads['b2']
        
    print(total_correct/np.float(len(x_train) ) )
time2 = time.time()
print(time2-time1)

0.8809166666666667
0.9389166666666666
0.95715
0.9655333333333334
0.9711666666666666
0.9766
0.98005
0.9804166666666667
0.9808
0.98225
0.9815166666666667
0.9810666666666666
0.9814333333333334
0.9823166666666666
0.9821
0.9818166666666667
0.98155
0.9819166666666667
0.98285
0.9818
0.9824666666666667
0.9822666666666666
0.9818666666666667
0.9829333333333333
0.9833
0.9819666666666667
0.9825833333333334
0.9825666666666667
0.9822333333333333
0.9822666666666666
0.9816666666666667
0.9824
0.9809666666666667
0.9821833333333333
0.981
0.9827833333333333
0.98215
0.9823
0.983
0.98245
378.89074182510376


In [4]:
######################################################
#test data
total_correct = 0
for n in range( len(x_test)):
    y = y_test[n]
    x = x_test[n][:]
    p = forward(x, y, model)
    prediction = np.argmax(p)
    if (prediction == y):
        total_correct += 1
        
#accuracy on test set
print(total_correct/np.float(len(x_test) ) )

0.972
